<a href="https://colab.research.google.com/github/Professor-Shin/CS-Class-Project/blob/main/Bioinformatics/FASTA_alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FASTA pairwise alignmet**

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.6 MB/s eta 0:00:00


### FASTA file
 - **code1.fasta** short nucleotide sequece
 - **code2.fasta** short protein sequence
 - **code3.fasta** long nucleotide sequece
 - **code4.fasta** long protein sequence

In [ ]:
filename = "/content/drive/MyDrive/sequence_file_example/code1.fasta"

In [ ]:
from Bio import SeqIO

def parse_fasta(filename):
    """Parses a FASTA file and returns a list of (id, sequence) tuples."""
    records = []
    for record in SeqIO.parse(filename, "fasta"):
        records.append((record.id, str(record.seq)))  #Ensure string conversion
    return records

# Example usage
sequences = parse_fasta(filename)
for seq_id, seq_string in sequences:
    print(f"Sequence ID: {seq_id}")
    print(f"Sequence: {seq_string}")
print(f"{sequences[0]}")

Sequence ID: sequence_1
Sequence: AGAGCTAGCTATC
Sequence ID: sequence_2
Sequence: AGAGCTGGGTACC
Sequence ID: sequence_3
Sequence: TCGATCGATCGA
Sequence ID: sequence_4
Sequence: TCGATCGATCGATC
('sequence_1', 'AGAGCTAGCTATC')


In [ ]:
from Bio.Align import substitution_matrices

def load_substitution_matrix(matrix_name):
    """Loads a substitution matrix from BioPython."""
    try:
        matrix = substitution_matrices.load(matrix_name)
        return matrix
    except Exception as e:
        print(f"Error loading matrix: {e}")
        return None

# Example Usage
blosum62 = load_substitution_matrix("BLOSUM62")
if blosum62 is not None:
    print(blosum62['A']['R'])  # Access score for Ala - Arg

-1.0


In [ ]:
print(blosum62)

#  Matrix made by matblas from blosum62.iij
#  * column uses minimum score
#  BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#  Blocks Database = /data/blocks_5.0/blocks.dat
#  Cluster Percentage: >= 62
#  Entropy =   0.6979, Expected =  -0.5209
     A    R    N    D    C    Q    E    G    H    I    L    K    M    F    P    S    T    W    Y    V    B    Z    X    *
A  4.0 -1.0 -2.0 -2.0  0.0 -1.0 -1.0  0.0 -2.0 -1.0 -1.0 -1.0 -1.0 -2.0 -1.0  1.0  0.0 -3.0 -2.0  0.0 -2.0 -1.0  0.0 -4.0
R -1.0  5.0  0.0 -2.0 -3.0  1.0  0.0 -2.0  0.0 -3.0 -2.0  2.0 -1.0 -3.0 -2.0 -1.0 -1.0 -3.0 -2.0 -3.0 -1.0  0.0 -1.0 -4.0
N -2.0  0.0  6.0  1.0 -3.0  0.0  0.0  0.0  1.0 -3.0 -3.0  0.0 -2.0 -3.0 -2.0  1.0  0.0 -4.0 -2.0 -3.0  3.0  0.0 -1.0 -4.0
D -2.0 -2.0  1.0  6.0 -3.0  0.0  2.0 -1.0 -1.0 -3.0 -4.0 -1.0 -3.0 -3.0 -1.0  0.0 -1.0 -4.0 -3.0 -3.0  4.0  1.0 -1.0 -4.0
C  0.0 -3.0 -3.0 -3.0  9.0 -3.0 -4.0 -3.0 -3.0 -1.0 -1.0 -3.0 -1.0 -2.0 -3.0 -1.0 -1.0 -2.0 -2.0 -1.0 -3.0 -3.0 -2.0 -4.0
Q -1.0  1.0  0.0  0.

### Information preparing

In [ ]:
match_score = int(input("Enter match score: "))
mismatch_score = int(input("Enter mismatch score: ")) # negative
gap_penalty = int(input("Enter gap penalty: ")) # negative

Enter match score: 2
Enter mismatch score: -1
Enter gap penalty: -1


In [ ]:
# Short squences (many sequnces in a fasta file)
seq1, seq2 = input("Enter 2 selected sequece number : ").split()
seq1, seq2 = sequences[int(seq1)-1], sequences[int(seq2)-1] # seq1, seq2 are coming from the same .fasta file (sequences set)

Enter 2 selected sequece number : 1 3


In [ ]:
# Long sequeces

### Local Alignment Algorithm

In [ ]:
def traceback(matrix, i, j, path, all_paths):
    if matrix[i][j][0] == 0:
        all_paths.append(path[:])
        return

    for direction in matrix[i][j][1]:
        if direction == 0:  # Diagonal
            ni, nj = i - 1, j - 1
        elif direction == 1:  # Up-left (separate from diagonal)
            ni, nj = i - 1, j - 1
        elif direction == 2:  # Left
            ni, nj = i, j - 1
        elif direction == 3:  # Up
            ni, nj = i - 1, j

        if 0 <= ni < len(matrix) and 0 <= nj < len(matrix[0]):
            traceback(matrix, ni, nj, [(ni, nj)] + path, all_paths)

In [ ]:
def linear_gap_local_alignment_revised(seq1, seq2, match_score, mismatch_score, gap_penalty):
    dp = []  # Scoring Matrix (zero matrix)
    for i in range(len(seq1[1]) + 1):
        dp.append([[0, []]] * (len(seq2[1]) + 1))

    max_score = -99
    max_score_indices = []  # Store all indices with max score
    print("Scoring Matrix")
    for i in range(0, len(seq1[1]) + 1):
        for j in range(0, len(seq2[1]) + 1):
            dp[i][j] = [0, []]
            if i > 0 and j > 0:
                score = [0, 0, dp[i][j - 1][0] + gap_penalty, dp[i - 1][j][0] + gap_penalty]
                if seq1[1][i - 1] == seq2[1][j - 1]:
                    score[1] = dp[i - 1][j - 1][0] + match_score
                elif seq1[1][i - 1] != seq2[1][j - 1]:
                    score[1] = dp[i - 1][j - 1][0] + mismatch_score

                dp[i][j][0] = max(score)
                for k in range(4):
                    if score[k] == dp[i][j][0]:
                        dp[i][j][1].append(k)

                # Check and update max_score_indices
                if dp[i][j][0] == max_score:
                    max_score_indices.append((i, j))
                elif dp[i][j][0] > max_score:
                    max_score = dp[i][j][0]
                    max_score_indices = [(i, j)]

            print(dp[i][j], end=" ")
        print()
    print(f"\nmax score : {max_score}")
    print(f"max score indices : {max_score_indices}")
    print("\n________________________________________________________________________________________________________")

    all_paths = []
    for index in max_score_indices:  # Traceback for each max score index
        traceback(dp, index[0], index[1], [index], all_paths)

    return max_score_indices, all_paths

### Visualize Result

In [ ]:
def visualize_alignment(seq1, seq2, paths):
    for path in paths:
        path = path[1:]
        print(f"path : {path}")
        aligned_seq1 = ""
        aligned_seq2 = ""
        match_pair = ""
        prev_index1, prev_index2 = 0, 0
        for seq_index in path:
            # Check if seq_index is a tuple and has at least 2 elements before accessing
            if isinstance(seq_index, tuple) and len(seq_index) >= 2:
                if seq_index[0] != prev_index1: # Aligned Sequence 1
                    # Access the sequence using seq1[1]
                    # assuming seq1 is a tuple (id, sequence)
                    aligned_seq1 += seq1[1][seq_index[0] -1]  # Adjust index for sequence access
                else:
                    aligned_seq1 += '-'

                if seq_index[1] != prev_index2: # Aligned Sequence 2
                    # Access the sequence using seq2[1]
                    # assuming seq2 is a tuple (id, sequence)
                    aligned_seq2 += seq2[1][seq_index[1] - 1]  # Adjust index for sequence access
                else:
                    aligned_seq2 += '-'

                # if seq1[1][seq_index[0] -1] == seq2[1][seq_index[1] - 1] :
                #     match_pair += '|'
                # else:
                #     match_pair += ' '

                prev_index1 = seq_index[0]
                prev_index2 = seq_index[1]
            else:
                print(f"Invalid seq_index: {seq_index}")

        print(f"{aligned_seq1}")
        # print(f"{match_pair}")
        print(f"{aligned_seq2}")

In [ ]:
# Only in class
match_score = int(input("Enter match score: "))
mismatch_score = int(input("Enter mismatch score: ")) # negative
gap_penalty = int(input("Enter gap penalty: ")) # negative
seq1 = ["seq1","TTGTXTAAG"]
seq2 = ["seq2","TXTXAAG"]

Enter match score: 1
Enter mismatch score: -1
Enter gap penalty: -3


In [ ]:
print(f"sequence 1 : {seq1[0]} - {seq1[1]}")
print(f"sequence 2 : {seq2[0]} - {seq2[1]}")
print()
max_score_index,all_paths = linear_gap_local_alignment_revised(seq1, seq2, match_score, mismatch_score, gap_penalty)
print()
visualize_alignment(seq1, seq2, all_paths)

sequence 1 : seq1 - TTGTXTAAG
sequence 2 : seq2 - TXTXAAG

Scoring Matrix
[0, []] [0, []] [0, []] [0, []] [0, []] [0, []] [0, []] [0, []] 
[0, []] [1, [1]] [0, [0]] [1, [1]] [0, [0]] [0, [0]] [0, [0]] [0, [0]] 
[0, []] [1, [1]] [0, [0, 1]] [1, [1]] [0, [0, 1]] [0, [0]] [0, [0]] [0, [0]] 
[0, []] [0, [0]] [0, [0, 1]] [0, [0]] [0, [0, 1]] [0, [0]] [0, [0]] [1, [1]] 
[0, []] [1, [1]] [0, [0]] [1, [1]] [0, [0]] [0, [0]] [0, [0]] [0, [0]] 
[0, []] [0, [0]] [2, [1]] [0, [0]] [2, [1]] [0, [0]] [0, [0]] [0, [0]] 
[0, []] [1, [1]] [0, [0]] [3, [1]] [0, [0, 2]] [1, [1]] [0, [0]] [0, [0]] 
[0, []] [0, [0]] [0, [0, 1]] [0, [0, 3]] [2, [1]] [1, [1]] [2, [1]] [0, [0]] 
[0, []] [0, [0]] [0, [0]] [0, [0]] [0, [0]] [3, [1]] [2, [1]] [1, [1]] 
[0, []] [0, [0]] [0, [0]] [0, [0]] [0, [0]] [0, [0, 3]] [2, [1]] [3, [1]] 

max score : 3
max score indices : [(6, 3), (8, 5), (9, 7)]

________________________________________________________________________________________________________

path : [(4, 1), (5, 2)